In [24]:
import os
import csv
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime



## Functions

In [95]:
def convert_date(input_date):
    formats = ["%d.%m.%y", "%d-%m-%Y", "%d-%b-%y"]
    target_format = "%Y-%m-%d"
    
    for date_format in formats:
        try:
            # Try parsing the input date using the current format
            parsed_date = datetime.strptime(input_date, date_format)
            # If successful, convert it to the target format
            result_date = parsed_date.strftime(target_format)
            return result_date
        except ValueError:
            pass
    
    # If none of the formats match, return an error message or handle as needed
    return "Invalid date format"

def date_difference(date1, date2):
    date_format = "%Y-%m-%d"
    
    try:
        # Parse the input dates
        parsed_date1 = datetime.strptime(date1, date_format)
        parsed_date2 = datetime.strptime(date2, date_format)
        
        # Calculate the difference in days
        difference = abs((parsed_date2 - parsed_date1).days)
        
        return difference
    except ValueError:
        return "Invalid date format"

def find_smallest_index(arr):


    if not arr:
        return "Array is empty"
    
    # Initialize variables to store the smallest number and its index
    smallest_number = arr[0]
    smallest_index = 0

    # Iterate through the array to find the smallest number and its index
    for i in range(1, len(arr)):
        if arr[i] < smallest_number:
            smallest_number = arr[i]
            smallest_index = i

    return smallest_index

def find_country_rank_and_code(country_name, match_date, rankings_data):
    match_date_rankings_data = convert_date(match_date)
    team_1_name = country_name
    if team_1_name in rankings_data['country_full'].unique():
        team1_rankings = rankings_data[rankings_data['country_full'] == team_1_name].reset_index()
    else:
        print(f'{team_1_name} not found in dataset')

    indexarray = []

    rank_date_array = []

    days_from_target_date_array = []

    for i in range(0, len(team1_rankings)):
        this_ranking_row_date = team1_rankings.at[i, 'rank_date']
        indexarray.append(i)
        rank_date_array.append(this_ranking_row_date)
        
        days_from_target_date = date_difference(match_date_rankings_data, this_ranking_row_date)

        days_from_target_date_array.append(days_from_target_date)

    date_of_closest_ranking = rank_date_array[find_smallest_index(days_from_target_date_array)]

    rankings_team1_closest_date = rankings_data[(rankings_data['rank_date'] == date_of_closest_ranking) & (rankings_data['country_full'] == team_1_name)].reset_index()
    team_1_country_rank = rankings_team1_closest_date.at[0, 'rank']
    team_1_country_points = rankings_team1_closest_date.at[0, 'total_points']



    return team_1_name, team_1_country_rank, team_1_country_points

## Results & Lineups

In [88]:
results = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/results_country_match_IDs_added.csv')
lineups = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/lineups_country_match_IDs_added.csv')


In [89]:
countrycodes = pd.read_csv("/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/countries_and_codes.csv")
rankings_data = pd.read_csv("/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv")

In [90]:
this_match_lineups = lineups[lineups['Match ID'] == 2650].reset_index()
this_match_results = results[results['Match ID'] == 2650] #2650

In [91]:
match_Date = this_match_lineups.at[0, 'Date']
#do this for zero 
team_1_name = this_match_lineups.at[0, 'TeamName']

team_2_name = this_match_lineups.at[1, 'TeamName']

In [93]:
find_country_rank_and_code(team_1_name, match_Date, rankings_data), find_country_rank_and_code(team_2_name, match_Date, rankings_data)

(('Croatia', 6, 1625.0), ('Wales', 23, 1514.0))

## Debugging

In [86]:
#team_1_name = 'Wales'
team_1_name = 'Croatia'


if team_1_name in rankings_data['country_full'].unique():
    team1_rankings = rankings_data[rankings_data['country_full'] == team_1_name].reset_index()

match_date_rankings_data = convert_date(match_Date)

indexarray = []
rank_date_array = []

days_from_target_date_array = []

for i in range(0, len(team1_rankings)):
    this_ranking_row_date = team1_rankings.at[i, 'rank_date']
    indexarray.append(i)
    rank_date_array.append(this_ranking_row_date)
    
    days_from_target_date = date_difference(match_date_rankings_data, this_ranking_row_date)

    days_from_target_date_array.append(days_from_target_date)

date_of_closest_ranking = rank_date_array[find_smallest_index(days_from_target_date_array)]

rankings_team1_closest_date = rankings_data[(rankings_data['rank_date'] == date_of_closest_ranking) & (rankings_data['country_full'] == team_1_name)].reset_index()
team_1_country_rank = rankings_team1_closest_date.at[0, 'rank']
team_1_country_points = rankings_team1_closest_date.at[0, 'total_points']


team_1_name, team_1_country_rank, team_1_country_points

('Croatia', 6, 1625.0)

In [84]:
rankings_team1_closest_date

,index,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,58308,23,Wales,WAL,1514.0,1539.0,4.0,UEFA,2019-06-14


In [61]:
#given a match date and a team 
#filter for that team name, and that date 

#matches are either in this form: '08.06.19'

#or 

#"14-01-2017"

#or "11-Nov-21"

#the form they need to be in to compare with rank date is 1992-12-31